## Install packages

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
     ابتدا packageهای لازم برای این تمرین را نصب می‌کنیم.
     </font>
</div>

In [1]:
# install libraries
# !pip install hazm
# !pip install pandas
# !pip install gensim
# !pip install tqdm
# !pip3 install sklearn
# !pip3 install seaborn
# !pip3 install networkx

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
     برای ساخت گراف به کمک networkx، به دلیل قدیمی بودن نسخه‌ی نصب شده با خطا مواجه شدیم که با این دستور و به‌روزرسانی آن، این مشکل حل شد.
     </font>
</div>

In [2]:
!pip3 install --upgrade scipy networkx

You should consider upgrading via the 'c:\users\mahdi\anaconda3\python.exe -m pip install --upgrade pip' command.


## Import libraries

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
     در این قسمت کتابخانه‌های مورد نیاز را import کردیم.
     </font>
</div>

In [1]:
# libraries
import codecs
import os
import json
import tqdm
import random
from collections import Counter
from nltk import FreqDist
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from gensim.models.fasttext import FastText
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import networkx as nx
from sklearn import preprocessing
from hazm import *

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         ابتدا مراحل تحلیل لینک را گام به گام انجام داده و در هر مرحله خروجی را بررسی و اصلاح می‌کنیم و سپس class به نام LinkAnalyser را ایجاد می‌کنیم.
         این کلاس به کمک تابع main_sentence_extractor جمله‌های مهم یک دسته از سندها را نشان می‌دهد. 
     </font>
</div>

## Load documents

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
     در این بخش، مجموعه داده دارای category که در تمرین چهارم از سایت نمناک جمع‌آوری کردیم را لود می‌کنیم. سپس دسته‌های موجود در این مجموعه داده را مشاهده
        می‌کنیم. سپس یکی از دسته‌ها (در اینجا دسته پیشگیری و بیماریها) را انتخاب می‌کنیم تا استخراج جملات مهم را بر روی سندهای این دسته انجام دهیم.
        به دلیل آنکه تعداد سندهای موجود برای برخی از دسته‌ها بسیار متفاوت از برخی دیگر است، از هر دسته حداکثر 500 سند را در نظر می‌گیریم و تحلیل لینک
        را بر روی این سندها انجام می‌دهیم.
     </font>
</div>

In [2]:
DIRNAME = '../new-dataset-with-category'
data = []
for i in range(1, 31):
    with open(f'{DIRNAME}/namnak-{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))

In [18]:
categories = set(item['category'] for item in data)

In [4]:
category = 'پیشگیری و بیماریها'
docs = [item for item in data if item['category'] == category]
docs = random.sample(docs, min(500, len(docs)))
docs_texts = [' '.join(doc['paragraphs']) for doc in docs]

In [5]:
len(docs)

500

## Instantiate hazm normalizer and lemmatizer

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         در این بخش، object مربوط به کلاس‌های Normalizer و Lemmatizer هضم را ایجاد می‌کنیم تا به کمک آن‌ها پیش‌پردازش‌های لازم
        را بر روی سندها انجام دهیم. 
     </font>
</div>

In [6]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()

 ## Stopwords

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         stopwordهایی که از کتابخانه هضم گرفتیم را به همراه stopwordهایی که خودمان اضافه کردیم، لود می‌کنیم و مجموعه این‌دو را به عنوان stopword نهایی در نظر
        می‌گیریم. پس از tokenize کردن جملات این stopwordها را از لیست توکن‌های جملات پاک می‌کنیم.
     </font>
</div>

In [7]:
# Persian Stopwords
# https://github.com/sobhe/hazm/blob/master/hazm/data/stopwords.dat
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

custom_stop_words = [normalizer.normalize(x.strip()) for x in codecs.open('custom_stopwords.txt','r','utf-8').readlines()]

total_stop_words = custom_stop_words + stopwords

# Preprocess documents

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         در متن اصلی، نیم‌فاصله‌ها با \u200f مشخص شده بود که ابزار لمتایز هضم آن را به عنوان نیم‌فاصله در نظر نمی‌گرفت به همین دلیل آن را با نیم‌فاصله جایگزین 
        کردیم.
     </font>
</div>

In [8]:
replaced_docs_texts = [doc_text.replace('-', ' ') for doc_text in docs_texts]
replaced_docs_texts = [doc_text.replace('\u200f', '‌') for doc_text in replaced_docs_texts]

## Sentence and word tokenizer

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
        سندها را بر اساس جمله و سپس بر اساس کلمه توکنایز کردیم به‌طوری که در words_list لیستی از لیست کلمات جمله‌ها داریم. همچنین جملات بسیار کوتاه و بسیار طولانی
        را حذف کردیم. ترشولد مربوط به مشخص کردن جملات بسیار کوتاه و بسیار طولانی را min_words_len_threshold و max_words_len_threshold مشخص 
        می‌کند.
    </font>
</div>

In [9]:
sentences = []
min_words_len_threshold = 4
max_words_len_threshold = 40

for doc_text in replaced_docs_texts:
    inner_sentences = sent_tokenize(normalizer.normalize(doc_text))
    sentences.extend([sentence for sentence in inner_sentences if min_words_len_threshold <= len(word_tokenize(sentence)) <= max_words_len_threshold])
words_list = [[_ for _ in word_tokenize(t)  if _ not in total_stop_words] for t in sentences]
words_list = [[lemmatizer.lemmatize(tok) for tok in ls] for ls in words_list]

# Create similarity matrix

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         در این قسمت ماتریس شباهت را ایجاد می‌کنیم. ابتدا یک ماتریس با ابعاد $sent num \times sent num$ با درایه‌های صفر ایجاد می‌کنیم.
        اگر تعداد کلمات مشترک میان جمله i و جمله j کوچک‌تر از threshold باشد، وزن یال بین راس‌های مربوط به این دو جمله را برابر صفر قرار می‌دهیم و در 
        غیر این صورت وزن را برابر $(intersect\:len / threshold)^{1.2} / len(word\:set)$ قرار می‌دهیم که intersect len تعداد کلمات مشترک بین دو جمله و 
        len(word set) برابر تعداد کلمات جمله i است.
        همچنین از آنجا که هر جمله تمام کلماتش با خودش یکسان است و شباهت بین یک جمله و خودش چندان معنادار نیست، درایه‌های روی قطر اصلی را صفر می‌کنیم.
     </font>
</div>

In [10]:
sent_num = len(words_list)
words_set = [set(ls) for ls in words_list]
similarity_mat = np.zeros((sent_num, sent_num), dtype=float)
threshold = 4

for i in tqdm.tqdm(range(sent_num)):
    for j in range(sent_num):
        intersect_len = len(words_set[i].intersection(words_set[j]))
        similarity_mat[i][j] = 0 if (intersect_len < threshold or i == j) else ((intersect_len / threshold) ** 1.2) / len(words_set[i])

100%|████████████████████████████████████████████████████████████████████████████| 18486/18486 [04:54<00:00, 62.80it/s]


## Normalize similarity matrix

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         حال ماتریس شباهت که در بالا ایجاد کردیم را به کمک sklearn.preprocessing، نرمال‌سازی می‌کنیم. 
     </font>
</div>

In [11]:
similarity_mat_normalized = preprocessing.normalize(similarity_mat, norm='l1')

## Create networkx graph

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         با استفاده از networkx گراف مربوط به ماتریس مجاورت را می‌سازیم. 
     </font>
</div>

In [12]:
graph = nx.from_numpy_array(similarity_mat_normalized)

# Implement page_rank algorithm

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         الگوریتم pagerank را برای گراف ایجاد شده پیاده‌سازی می‌کنیم. 
     </font>
</div>

In [13]:
page_rank = nx.pagerank(graph, alpha=0.9)

In [14]:
page_rank

{0: 7.81962059822684e-06,
 1: 0.0002026810589839203,
 2: 7.81962059822684e-06,
 3: 7.010556067162327e-05,
 4: 7.81962059822684e-06,
 5: 9.447818982835023e-06,
 6: 7.81962059822684e-06,
 7: 4.944871984153406e-05,
 8: 7.81962059822684e-06,
 9: 6.86059440878567e-05,
 10: 6.86059440878567e-05,
 11: 7.81962059822684e-06,
 12: 9.128373705860303e-05,
 13: 1.9875916922259703e-05,
 14: 5.001433150665927e-05,
 15: 5.149283573417636e-05,
 16: 7.81962059822684e-06,
 17: 8.945290162178061e-05,
 18: 7.81962059822684e-06,
 19: 8.817809986061927e-05,
 20: 7.81962059822684e-06,
 21: 0.0003079140712681693,
 22: 0.0001981221788880568,
 23: 8.181337887357108e-05,
 24: 1.4412737391631227e-05,
 25: 7.81962059822684e-06,
 26: 7.81962059822684e-06,
 27: 0.0001401404718207084,
 28: 0.00010325829026468594,
 29: 6.53300133687178e-05,
 30: 4.643973758209722e-05,
 31: 9.212042089940889e-05,
 32: 1.1268080612379086e-05,
 33: 9.989729317425719e-05,
 34: 5.8204683125633847e-05,
 35: 7.191129384019005e-05,
 36: 0.0001

## Print top n sentences of document

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         با استفاده از تابع get_top_n، مهم‌ترین n جمله‌ای به کمک الگوریتم pagerank به دست آمده است را نشان می‌دهیم. 
     </font>
</div>

In [15]:
def get_top_n(values, n=5):
    np_page_rank = np.array(list(values))
    top_n = np_page_rank.argsort()[-n:][::-1]
    return top_n

In [16]:
get_top_n(page_rank.values())

array([1955,   96,  863,  924,  255], dtype=int64)

In [17]:
for i in get_top_n(page_rank.values()):
    print(sentences[i])
    print('---------------')

شما همچنین می‌توانید ۱ قاشق غذا خوری از سرکه سیب و عسل را در یک لیوان آب گرم مخلوط کنید و آن را دو بار در روز برای ۱ یا ۲ هفته بنوشید.
---------------
ورزش به کنترل وزن، کاهش فشار خون، کاهش کلسترول و تری گلیسیرید LDL مضر، افزایش کلسترول HDL سالم، تقویت عضلات و استخوان‌ها، کاهش اضطراب و بهبود سلامت عمومی کمک می‌کند.
---------------
زمانی که علائم و نشانه‌های مشکلات روده‌ای را داشته باشید: این آزمایش به پزشک شما کمک می‌کند که علت‌های درد شکم، خونریزی مقعد، یبوست مزمن، اسهال مزمن یا دیگر مشکلات روده را تشخیص دهد.
---------------
اگر کمر درد به دلیل چنین مشکلاتی رخ داده باشد، احتمالا به قسمت‌های پایینی بدن هم انتقال پیدا می‌کند اما اگر علت آن کرونا باشد، چنین اتفاقی نخواهد افتاد و معمولا این درد به پا‌ها نمی‌رسد.
---------------
تحقیقات نشان داده است که ماساژ شکمی میتواند در درمان یبوست مزمن موثر باشد زیرا میزان حرکات روده را افزایش می‌دهد، باعث کاهش زمان انتقال مدفوع از روده می‌شود و به درد و ناراحتی شکم هنگام یبوست کمک می‌کند.
---------------


<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
         در این قـسمت الـگوریـتم hits پـیاده‌سـازی شـده اسـت کـه hubs و authorities را خـروجـی مـی‌دهـد. 
     </font>
</div>

# Implement hits algorithm

In [19]:
hubs,authorities = nx.hits(graph)

In [20]:
get_top_n(hubs.values(), n=10)

array([ 4013,  4029,  4526, 14428, 17029, 18362, 15424,  5839, 14031,
        7067], dtype=int64)

In [21]:
get_top_n(authorities.values(), n=10)

array([ 4013,  4029,  4526, 14428, 17029, 18362, 15424,  5839, 14031,
        7067], dtype=int64)

## Print top n sentences of document

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
  در این بخش خروجی hubs و authorities نشان داده شده است.
     </font>
</div>

### Hubs

In [22]:
for i in get_top_n(hubs.values()):
    print(sentences[i])
    print('---------------')

سن: بیماری روماتیسم می‌تواند جوانان را در هر سنی درگیر کندو بیشتر افراد مبتلا به آن بین ۴۰ تا ۶۰ سال سن دارند و تقریبا سه چهارم افراد مبتلا به روماتیسم حین انجام شغلشان متوجه این بیماری شده‌اند.
---------------
سن شیوع روماتیسم همه سنین می‌تواند درگیر بیماری روماتیسم شود؛ این بیماری حتی کودکان و نوزادان را درگیر می‌کند.
---------------
در حقیقت اگر به طور کلی بخواهید ببینید چه کسی به چه بیماری مبتلا شده متوجه خواهید شد که برخی از بیماری­‌ها بر آقایان بیشتر از بانوان تاثیر می‌گذارد.
---------------
دسته بندی‌های ریسک یا پرخطر شامل افراد سیگاری است که تقریبا بین ۴۰ تا ۷۵ سال سن دارند.
---------------
بسیاری از اشخاص مبتلا به این بیماری ۷۲ سال یا بیش‌تر از آن سن دارند.
---------------


### Authorities

In [23]:
for i in get_top_n(authorities.values()):
    print(sentences[i])
    print('---------------')

سن: بیماری روماتیسم می‌تواند جوانان را در هر سنی درگیر کندو بیشتر افراد مبتلا به آن بین ۴۰ تا ۶۰ سال سن دارند و تقریبا سه چهارم افراد مبتلا به روماتیسم حین انجام شغلشان متوجه این بیماری شده‌اند.
---------------
سن شیوع روماتیسم همه سنین می‌تواند درگیر بیماری روماتیسم شود؛ این بیماری حتی کودکان و نوزادان را درگیر می‌کند.
---------------
در حقیقت اگر به طور کلی بخواهید ببینید چه کسی به چه بیماری مبتلا شده متوجه خواهید شد که برخی از بیماری­‌ها بر آقایان بیشتر از بانوان تاثیر می‌گذارد.
---------------
دسته بندی‌های ریسک یا پرخطر شامل افراد سیگاری است که تقریبا بین ۴۰ تا ۷۵ سال سن دارند.
---------------
بسیاری از اشخاص مبتلا به این بیماری ۷۲ سال یا بیش‌تر از آن سن دارند.
---------------


## check similarity matrix symmetricity

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
  در این قسمت بـررسـی و تـایـید شـده اسـت کـه مـاتـریـس مـتقارن نیسـت. بـا ایـن
کــار اطــمینان حــاصــل مــیشــود کــه خــروجــی hubs و authorities حــتی اگــر کــامــلا یــکسان بــاشــد بــه دلــیل مــتقارن بــودن  ماتریس نیست. دلیل مشابهت زیاد این دو خروجی در ادامه بیان شده است.
     </font>
</div>

In [24]:
np.allclose(similarity_mat_normalized, similarity_mat_normalized.T, atol=1e-2)

False

## Link analyser final class

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
  در این بخش تمام مراحلی که تا اینجا گام به گام پیش رفتیم را در کنار هم قرار دادیم و این کلاس را ایجاد کردیم. در init این کلاس یک دیتاست می‌گیرد که شامل سندها و دسته‌هایشان است و در توابع دیگر کلاس استفاده می‌شود. این کلاس دارای تابع main_sentence_extractor هست که یک دسته به عنوان ورودی می‌گیرد، متد را می‌گیرد که می‌تواند page rank یا الگوریتم hits باشد، تعداد جملاتی که قرار است در خروجی نشان داده شود را هم به عنوان ورودی می‌گیرد و تعداد سندهایی که قرار است از میان آن‌ها جملات را extract کند را هم می‌گیرد.
     این تابع بر اساس این ورودی‌ها به تعداد مشخص‌شده سند از آن موضوع را برمی‌دارد و الگوریتم‌ را رویش اجرا می‌کند و در آخر تعداد n جمله‌ی top و bottom را چاپ می‌کند.
     </font>
</div>

In [26]:
class LinkAnalyser:
    
    def __init__(self, dataset, threshold=4, min_words_len_threshold=4, max_words_len_threshold=40):
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()
        self.threshold = threshold
        self.tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words=None, ngram_range=(1,1))
        self.min_words_len_threshold = min_words_len_threshold
        self.max_words_len_threshold = max_words_len_threshold
        self.dataset = dataset
    
        stopwords = [self.normalizer.normalize(x.strip()) for x in codecs.open('../stopwords/stopwords.txt','r','utf-8').readlines()]
        custom_stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('../stopwords/custom_stopwords.txt','r','utf-8').readlines()]
        self.total_stop_words = custom_stop_words + stopwords
        
    def get_top_n(self, values, n=5, descending=True):
        np_page_rank = np.array(list(values))
        if descending:
            top_n = np_page_rank.argsort()[-n:][::-1]
        else:
            top_n = np_page_rank.argsort()[:n]
        return top_n
    
    def pprint(self, ranking, sentences, n=5, descending=True):
        for i in self.get_top_n(ranking.values(), n=n, descending=descending):
            print(sentences[i])
            print('---------------')
    
    def main_sentence_extractor(self, category, method='pr', n=5, number_of_samples=500):
        # method can be pr or hits
        
        docs = [item for item in self.dataset if item['category'] == category]
        docs = random.sample(docs, min(number_of_samples, len(docs)))
        docs_texts = [' '.join(doc['paragraphs']) for doc in docs]
        
        replaced_docs_texts = [doc_text.replace('-', ' ') for doc_text in docs_texts]
        replaced_docs_texts = [doc_text.replace('\u200f', '‌') for doc_text in replaced_docs_texts]
    
        sentences = []

        for doc_text in replaced_docs_texts:
            inner_sentences = sent_tokenize(self.normalizer.normalize(doc_text))
            sentences.extend([sentence for sentence in inner_sentences if self.min_words_len_threshold <= len(word_tokenize(sentence)) <= self.max_words_len_threshold])
        words_list = [[_ for _ in word_tokenize(t)  if _ not in self.total_stop_words] for t in sentences]
        words_list = [[self.lemmatizer.lemmatize(tok) for tok in ls] for ls in words_list]
        
        sent_num = len(words_list)
        words_set = [set(ls) for ls in words_list]
        similarity_mat = np.zeros((sent_num, sent_num), dtype=float)
        for i in tqdm.tqdm(range(sent_num)):
            for j in range(sent_num):
                intersect_len = len(words_set[i].intersection(words_set[j]))
                similarity_mat[i][j] = 0 if (intersect_len < self.threshold or i == j) else ((intersect_len/self.threshold) ** 1.2) / len(words_set[i]) / len(words_set[i])
    
        similarity_mat_normalized = preprocessing.normalize(similarity_mat, norm='l1')
        graph = nx.from_numpy_array(similarity_mat_normalized)
        
        result = {}
        
        if method == 'pr':
            ranking = nx.pagerank(graph, alpha=0.9)
            print('Page rank:')
            print(f'top {n}')
            result['tops'] = []
            for i in self.get_top_n(ranking.values(), n=n, descending=True):
                result['tops'].append(sentences[i])
            self.pprint(ranking, sentences, n=n)
            print(f'bottom {n}')
            result['bottoms'] = []
            for i in self.get_top_n(ranking.values(), n=n, descending=False):
                result['bottoms'].append(sentences[i])
            self.pprint(ranking, sentences, n=n, descending=False)
        else:
            hubs, authorities = nx.hits(graph)
            result['Authorities'] = {'tops': [], 'bottoms':[]}
            result['Hubs'] = {'tops': [], 'bottoms':[]}
            print('Authorities:')
            print(f'top {n}')
            for i in self.get_top_n(authorities.values(), n=n, descending=True):
                result['Authorities']['tops'].append(sentences[i])
            self.pprint(authorities, sentences, n=n)
            print(f'bottom {n}')
            for i in self.get_top_n(authorities.values(), n=n, descending=False):
                result['Authorities']['bottoms'].append(sentences[i])
            self.pprint(authorities, sentences, n=n, descending=False)
            print('Hubs:')
            print(f'top {n}')
            for i in self.get_top_n(hubs.values(), n=n, descending=True):
                result['Hubs']['tops'].append(sentences[i])
            self.pprint(hubs, sentences, n=n)
            print(f'bottom {n}')
            for i in self.get_top_n(hubs.values(), n=n, descending=False):
                result['Hubs']['bottoms'].append(sentences[i])
            self.pprint(hubs, sentences, n=n, descending=False)
        return result


## Instantiate LinkAnalyser

In [27]:
link_analyser = LinkAnalyser(data)

In [30]:
data = {}
for category in categories:
    inner_data = {}
    inner_data['page_rank'] = link_analyser.main_sentence_extractor(category, method='pr', n=3)
    inner_data['hits'] = link_analyser.main_sentence_extractor(category, method='hits', n=3)
    data[category] = inner_data

with open(f'../models/link_analyser_categories.json', 'w', encoding="utf-8") as f:
    json.dump(data, f)

100%|█████████████████████████████████████████████████████████████████████████████| 9681/9681 [01:00<00:00, 160.00it/s]


Page rank:
top 3
جوش شیرین به عنوان شامپو خشک: اگر پوست سرتان خیلی چرب است و زمانی برای شستن موها ندارید، می‌توانید به سادگی از جوش شیرین استفاده کنید و مصرف محصولات مضر حاوی مواد شیمیایی را کاهش دهید.
---------------
برای اینکه بتوانید در طول روز انرژی بیشتری داشته باشید و انرژی خود را صرف کارهای خلاقانه کنید لازم است از میوه جات و سبزیجات زیادی در رژیم غذاییتان استفاده کنید تا تاثیرات آن را روی بدن تان ببینید.
---------------
درست است که هفته‌ای یک بار دویدن و روزی نیم ساعت پیاده رفتن عمر شما را طولانی می‌کند اما محققان می‌گویند اگر تنها ۳۰دقیقه زمان ثابت را به ورزش کردن اختصاص دهید، عمرتان ۱۵سال طولانی‌تر می‌شود.
---------------
bottom 3
به تعویق انداختن زمان بیولوژیک بدن، از لحاظ علمی امکان پذیر است.
---------------
پیاده روی بهترین روش فعالیت کردن برای افراد سالمند است.
---------------
این در حالی است که بی تحرکی برای سلامت آنها بسیار مضر بوده و باید ساعاتی را از منزل خارج شوند.
---------------


100%|█████████████████████████████████████████████████████████████████████████████| 9681/9681 [01:00<00:00, 159.47it/s]


Authorities:
top 3
جایگزین کردن مواد شوینده با جایگزین کردن و استفاده از مواد شوینده دوستدار محیط زیست می‌توانید میزان آلودگی هوای داخل خانه را کاهش داده، خطر ابتلا به بیماریهای آسم و آلرژی را کم کنید.
---------------
این کار ساده از انتشار آلودگی‌های بیرون مانند آلودگی‌های ناشی از خودروها، مواد شیمیایی و افت کش‌ها در داخل خانه جلوگیری می‌کند.
---------------
فراموش نکنید که اصلا از جاروبرقی برای تمیز کردن محل استفاده نکنید چراکه باعث پخش آلودگی در هوای محیط شود.
---------------
bottom 3
عامل مشخصی خواب شما را مختل می‌کند: صدای دزدگیر ماشین همسایه، فرزندی که در حال دندان درآوردن است، استفاده از داروهای استروئیدی، نگرانی‌های شغلی و … هیچ کس دوست ندارد خوابش را از دست بدهد.
---------------
این متخصص داخلی در پایان از همه افراد خواست تا با توجه به این مسئله بیش از پیش هوای پدران را داشته و سعی کنند در زمینه استفاده از کولر شرایط فیزیولوژیکی بدن آنها را در نظر بگیرند.
---------------
رنگ لعابی پودری را روی نشاسته بپاشید و بگذارید کودک ترکیب رنگ‌‌ها را تجربه کند.
---------------
Hubs:
top 3

100%|███████████████████████████████████████████████████████████████████████████| 12989/12989 [01:49<00:00, 118.24it/s]


...

Authorities:
top 3
شوره دانه‌های سفید و خشک پوست می‌باشد که حین شانه زدن، روی لباس و شانه‌های شما می‌ریزد و هرگز ضرری ندارد اما باعث خارش سر می‌شود و گاهی اوقات عاملی برای خجالت کشیدن است.
---------------
اگر سرتان خارش دارد و به طور مرتب شوره هایی را روی شانه هایتان پیدا می‌کنید, ممکن است شوره سر داشته باشید.
---------------
زمانی که پوست سر خود را می‌خارانید، روی شانه‌ها و کمر شما می‌ریزند و باعث خجالت می‌شوند.
---------------
bottom 3
اگزمای آتوپیک چیست و چه علل و علائمی در کودکان دارد؟
---------------
همین امر باعث تغییر شکل ستون فقرات شما می‌شود و به مرور زمان یک موقعیت آسیب رسان برای بدن شما خواهد بود.
---------------
همانطور که می‌دانید ستون فقرات شما به صورت منحنی S شکل متعادلی است که می‌تواند شما را پایدار و ثابت نگه دارد.
---------------
Hubs:
top 3
شوره دانه‌های سفید و خشک پوست می‌باشد که حین شانه زدن، روی لباس و شانه‌های شما می‌ریزد و هرگز ضرری ندارد اما باعث خارش سر می‌شود و گاهی اوقات عاملی برای خجالت کشیدن است.
---------------
زمانی که پوست سر خود را می‌خارانید، روی شانه

100%|███████████████████████████████████████████████████████████████████████████| 10045/10045 [01:09<00:00, 144.43it/s]


Page rank:
top 3
بهتر است: بعد از خوردن این خوراکی‌ها، ۴۰ دقیقه برای مسواک زدن صبر کنید و اگر می‌خواهید بلافاصله دندان‌های تان را تمیز کنید، با چرخاندن آب خالی در دهان تان این کار را انجام دهید.
---------------
در بخش سلامت دهان و دندان در نمناک خواندید که خشکی دهان ممکن است منجر به پوسیدگی دندان شود به همین دلیل بیماران دیابتی برای افزایش تولید بزاق باید آب زیادی بنوشند.
---------------
ایمپلنت دندان برعکس دندان طبیعی در معرض آسیب دیدگی، بیماری، پوسیدگی و مشکلات لثه قرار نمی‌گیرد بنابراین طول عمر بالایی دارد.
---------------
bottom 3
مطمئن شوید که تمام نشانه‌های عفونت از بین رفته است.
---------------
مسواک‌های که با باتری کار می‌کنند: این نوع مسواک‌ها قابل حمل و ارزان بوده اما عمر کوتاهی دارند.
---------------
کیست همان عفونت است؟
---------------


100%|███████████████████████████████████████████████████████████████████████████| 10045/10045 [01:09<00:00, 145.38it/s]


Authorities:
top 3
اگر فضای دندان از دست رفته خالی بماند یا جایگزین مناسبی برای آن استفاده نشود، مشکلاتی مانند تحلیل و کاهش استخوان فک ایجاد میشود.
---------------
اگر استرس درمان نشود، می‌تواند منجر به مشکلات جدی‌تری مانند افسردگی و یا شکستگی استخوان و یا حتی از دست دادن استخوان شود.
---------------
کیست‌ها اگر درمان نشوند چون حفره‌های خالی از استخوان در فک هستند، استخوان فک را در برابر ضربه تضعیف می‌کنند.
---------------
bottom 3
بنابراین اگر نمی‌خواهید در یک جلسه‌ی کاری یا آشنایی دچار چنین مصائبی شوید بهتر است مصرف کلم‌ها را به موقعیت دیگری موکول کنید.
---------------
او ادامه داد: مغز ما به طور طبیعی برای بازسازی سلول‌ها نیاز به سوخت دارد که این سوخت از چربی و شکر استفاده می‌کند.
---------------
الکل باعث تعریق می‌شود عرقی که از طریق غدد درم ترشح می‌شود برای تنظیم دمای بدن است.
---------------
Hubs:
top 3
اگر فضای دندان از دست رفته خالی بماند یا جایگزین مناسبی برای آن استفاده نشود، مشکلاتی مانند تحلیل و کاهش استخوان فک ایجاد میشود.
---------------
اگر استرس درمان نشود، می‌تواند منج

## Word similarity with page rank method

In [27]:
link_analyser.main_sentence_extractor('تغذیه', method='pr', n=5, number_of_samples=200)

100%|██████████| 8693/8693 [00:56<00:00, 153.25it/s]


Page rank:
top 5
کدو تنبل منبع خوبی از ویتامین C است که یک آنتی اکسیدان قوی است و همچنین حاوی بتاکاروتن است که به جلوگیری از آسیب UV و بهبود بافت پوست کمک می‌کند.
---------------
بارها در مطالب مختلف بخش سلامت نمناک عنوان کرده‌ایم که وجود ویتامین‌های A و C در آب هویج زنجبیل باعث ایجاد تغییرات قابل توجهی در مغز استخوان شده که باعث بهبود سیستم ایمنی بدن نیز خواهد شد.
---------------
اگر می‌خواهید وزن بدن خود را کاهش دهید و یک رژیم غذایی خوب داشته باشید، خوردن نارنگی یکی از بهترین انتخاب‌ها است، زیرا این میوه، مانند دیگر میوه‌ها، حاوی مقدار زیادی فیبر است.
---------------
مقدار انسولین تولید شده در بیماران دیابتی به اندازه کافی نیست و تحقیقات نشان می‌دهند که استفاده از گل میخک، به تولید انسولین در بدن و همچنین حفظ سطح قند خون کمک می‌کند.
---------------
در حقیقت، پیازچه از زمان قدیم توسط پدران ما به عنوان داروی گیاهی برای درمان تورم پا استفاده شده است این، به آن دلیل است که پیازچه دارای خواص ضد التهابی است، و تورم را کاهش میدهد.
---------------
bottom 5
ابتدا مقداری دنبه تازه گوسفندی را ب

<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
  در مثال بالا دیده می‌شود که دسته‌ی انتخاب‌شده تغذیه است. جملاتی که به عنوان top خروجی داده شده واقعا به موضوع تغذیه و علم تغذیه مربوط است و به نطر می‌رسد که خروجی خوبی گرفته‌ایم. هم‌چنین جملاتی که به عنوان bottom داده شده‌اند نیز اطلاع مفیدی به ما نمی‌دهند و جملات مرتبطی نیستند.
     </font>
</div>

In [28]:
link_analyser.main_sentence_extractor('سلامت جنسی', method='pr', n=5, number_of_samples=500)

100%|██████████| 12916/12916 [02:10<00:00, 99.30it/s] 


Page rank:
top 5
ویاگرا یکی دیگر از داروهای درمان ناتوانی جنسی در مردان است و متعلق به خانواده داروهای بازدارنده فسفودی استراز ۵ است که جریان خون در آلت تناسلی را افزایش می‌دهد و اختلال نعوظ را درمان می‌کند.
---------------
برقراری یک رابطه جنسی سالم و رسیدن به ارگاسم مناسب اثرات مثبت بسیاری را برای جسم و روح زن و مرد خواهد داشت، از زیبایی جسم تا آرامش روح همه را می‌تواند با داشتن رابطه زناشویی مفید کسب کرد.
---------------
برای برقراری رابطه جنسی با همسر بی حوصله تان ابتدا طوری وانمود کنید که می‌خواهید کمی با او شوخی داشته باشید، در این صورت همسرتان مخالفت نخواهد کرد و شما می‌توانید مدت زمان بیشتری را به تحریک او اختصاص دهید.
---------------
تخمدان‌ها بعد از یائسگی مقدار کمی هورمون تولید می‌کنند اما میزان پروژسترون و استروژن پایدار باقی نمی‌ماند و سطح پایین استروژن باعث کاهش میل جنسی می‌شود.
---------------
مصرف سیر را افزایش دهید: سیر در تقویت سیستم ایمنی بدن، کاهش فشار خون، کاهش قند خون و مبارزه با سرطان کمک می‌کند سیر، غذایی است که برای بیماران مبتلا به زگیل‌های مقعد مناسب است.
---

## Word similarity with hits method

In [29]:
link_analyser.main_sentence_extractor('سلامت روان', method='hits', n=3, number_of_samples=500)

100%|██████████| 13799/13799 [02:39<00:00, 86.35it/s]
C:\Users\ASUS\anaconda3\lib\site-packages\networkx\algorithms\link_analysis\hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


Authorities:
top 3
اگر در خوابیدن مشکل دارید، ۲۰ دقیقه قبل از خواب مدیتیشن انجام دهید تا بدن تان ریلکس شود و مطمئن باشید که صبح زودتر از خواب بیدار می‌شوید.
---------------
اگر به مدت چند دقیقه در آفتاب قدم بزنید, حتی بهتر خواهید بود, چون سیگنال هایی به بدن شما ارسال می‌کند تا بیدار شوید.
---------------
بسیاری از ما صبح‌ها بیدار می‌شویم و قهوه تازه می‌نوشیم و از این به بعد می‌توانیم روتین صبحگاهی را تغییر دهیم و چیز متفاوتی را امتحان کنیم.
---------------
bottom 3
ب) از کودک سوال کنید (بعد از اینکه آن را گره زد)، آیا هوای آن بادکنک را می‌تواند خارج کند یا هوایی به داخل آن وارد کند.
---------------
از کودک بپرسید اگر مغزش بادکنک باشد و خشم هوای درون آن باشد، آیا جایی برای فکر کردن واضح و روشن خواهد داشت.
---------------
ب) برای کودک توضیح دهید که اگر بادکنک یک آدم باشد، ترکیدن بادکنک یک عمل پرخاشگرانه مثل زدن یک فرد، یا یک شی خواهد بود.
---------------
Hubs:
top 3
اگر در خوابیدن مشکل دارید، ۲۰ دقیقه قبل از خواب مدیتیشن انجام دهید تا بدن تان ریلکس شود و مطمئن باشید که صبح زودتر از خواب 

In [30]:
link_analyser.main_sentence_extractor('دهان و دندان', method='hits', n=3, number_of_samples=250)

100%|██████████| 7145/7145 [00:43<00:00, 165.55it/s]
C:\Users\ASUS\anaconda3\lib\site-packages\networkx\algorithms\link_analysis\hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


Authorities:
top 3
رایج‌ترین رویکردی که افراد برای ضد عفونی کردن مسواک به آن متکی هستند استفاده از دهان شویه‌ای مانند لسترین است بدان صورت که رشته‌های موئی مسواک را به مدت ۳ ۵ دقیقه درون محلول دهان شویه قرار دهید.
---------------
سپس رشته‌های موئی مسواک را به مدت ۲۰ ۳۰ دقیقه دراین محلول بخیسانید.
---------------
از طرفی باید مراقب باشید زیرا قرارگیری بیش از حد مسواک درون دهان شویه سبب آسیب رساندن به رشته‌ها خواهد شد.
---------------
bottom 3
برای این درد، می‌توانید داروی ضد دردی که توسط جراح به شما تجویز شده را مصرف کنید.
---------------
لذا پزشک معمولا از دوز کم دارو را تجویز کرده و سپس با خفیف شدن عوارض جانبی، به تدریج دوز آن را افزایش می‌دهد.
---------------
فضایی مثبت را در ذهن او بسازید اگر اولین بار است که کودک خود را نزد دندانپزشکی می‌برید، حتما با عبارات مثبت به شرح و توضیح در مورد دندانپزشکی، مهربان بودن دندانپزشک و آسان بودن اقدامات بپردازید.
---------------
Hubs:
top 3
رایج‌ترین رویکردی که افراد برای ضد عفونی کردن مسواک به آن متکی هستند استفاده از دهان شویه‌ای مانند لسترین ا

# تحلیل نتایج
<div dir = "rtl" style="direction:rtl;line-height:250%;">
	<font face="XB Zar" size=4>
     بــه طــور کــلی شــانــس حــضور جــملات طــولانــی در مــیان جــملات top بیشتر است که منطقی هم هست چرا که از سویی جمله‌ی مـهم جمله‌ای است که اطلاعات بیشتری در آن باشد. در جملات bottom نیز معمولا جملات کوتاه کـه اطـلاعـات خاصی ندارند حاضر شده‌اند.
     به طور کلی با آزمایش روش‌های مختلف، از برای ساختن ماتریس مشابهت گرفته تا انتخاب ضرایب، مـــقادیر و روش‌هـــای بـــهینه‌تـــر اســـتفاده شده‌اند تا جملاتی کـه اطـلاعات مـــرتـــبط زیـــادی در بر دارنـــد و مـــهم
هستند خروجی داده شود.  

هم‌چنین نکته‌ای که قابل توجه است این است که اگر یک سند بیش از حد طولانی باشد (تعداد جملاتش به نسبت جملات کل یک دسته زیاد باشد)، چون جملات آن سند با  هم ارتباط زیادی دارند احتمالا شانس بیش‌تری برای حضور در جملات top دارند و از آنجایی که ما سندها را بر اساس جملات شکانده‌ایم این مسئله می‌تواند مشکل‌ساز باشد. 

به علاوه همانطور که می‌بینیم، جملاتی که در قسمت top آمده‌اند معمولا کلمه‌هایی دارند که داخل آن دسته کلمه‌های عمومی هستند و زیاد تکرار شده‌اند به همین خاطر الگوریتم ما آن جملات را محوری و مهم تشخیص می‌دهد. بر عکس اغلب کلماتی را در جملات bottom داریم که یک سری کلمات خاص دارند که فقط در جاهای محدودی استفاده شده‌اند و اطلاعات زیادی ندارند.

تــوجــه: در بــخش hubs و authorities تــوضــیح دادیــم کــه بــرای یــکسان نشــدن خــروجــی hubs و authorities ماتــریـس بایـد نـامـتقارن مـی‌بـود کـه حـتی ایـن مـسئله بـررسـی و تـایـید شـد. امـا دلـیل ایـنکه خـروجـی ایـن دو مـخصوصـا در جـملات top یـکسان بـه نـظر مـی‌رسـد ایـن اسـت کـه مـاتـریـس شـباهـت بـر اسـاس تـعداد اشـتراک کـلمات مـیان جـملات مـختلف تـعریـف شـده اســت و وقــتی جــمله‌ی a بــا b اشــتراک زیــادی دارد، مــنطقا b نــیز بــا a اشــتراک زیــادی خــواهــد داشــت. حــال هــمانــطور کــه مــی‌دانــیم hubs و authorities بــرای ارجــاع تــعریــف مــی‌شــونــد یــعنی یــکی بــیانــگر جـملاتی اسـت که خیلی به آن ارجاع داده مـی‌شـود و دیـگری بـیانـگر جـملاتـی اسـت کـه زیـاد بـه بـقیه ارجـاع مـی‌دهـند. از آنـجایـی کـه در تـعریـف مـاتـریـس و گـراف ســـاخـــته شـــده، ایـــن ارجـــاع هـــمان اشـــتراک اســـت و اشـــتراک هـــم دو ســـویـــه اســـت، خـــروجـــی hubs و authorities نســـبتا مشابه شده است.
     </font>
</div>